# Codigo sacado de Eugenio Negrin

In [1]:
!pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [4]:
!gsutil cp /home/fedepicado/buckets/b1/datasets/competencia_02_DQ_ft.parquet /home/fedepicado/datasets/

Copying file:///home/fedepicado/buckets/b1/datasets/competencia_02_DQ_ft.parquet...
- [0 files][    0.0 B/  2.8 GiB]                                                / [1 files][  2.8 GiB/  2.8 GiB]                                                
Operation completed over 1 objects/2.8 GiB.                                      


In [5]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
dataset_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
dataset_file = 'competencia_02_DQ_ft.parquet'
full_path = dataset_path + dataset_file

In [11]:
#semillas
semillas = [540079,250829,314299,302111,801007]

#meses entrenamiento
meses_train = [202012,202101,202102,202103,202104]

In [7]:
data = pd.read_parquet(dataset_path + dataset_file)

In [8]:
df=data.copy()
df.shape

(4735593, 288)

In [9]:
df['clase_binaria'] = 0
df['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [ ]:
df = df[df['foto_mes'].isin(meses_train)]

In [ ]:
df_prop = (
    df.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

df_prop.columns = ['foto_mes', 'proportion_0', 'proportion_1']
df_prop

In [ ]:
df_subsampled = []

for mes, group in df.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
data = pd.concat(df_subsampled, ignore_index=True)